In [1]:
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import time
import mysql.connector
from mysql.connector import Error
import ast

In [71]:
myhref = []
num_pages = 20  
retry_delay = 5  

for page_num in tqdm(range(1, num_pages + 1)):  
    for _ in range(3): 
        try:
            url = f"https://www.transfermarkt.us/spieler-statistik/wertvollstespieler/marktwertetop/plus/ajax/yw1/ausrichtung/Sturm/spielerposition_id/alle/altersklasse/alle/jahrgang/0/land_id/0/kontinent_id/6/yt0/Show/0//page/{page_num}"
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser")

            Link1 = Soup.find("div", {'class': "responsive-table"})
            if Link1:  # Check if the div is found
                links = Link1.find_all('a')
                for a in links:
                    href = a.get('href')
                    if href and "profil/spieler/" in href:
                        myhref.append(href)
                break  
        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)

100%|███████████████████████████████████████████| 20/20 [00:42<00:00,  2.10s/it]


In [72]:
len(myhref)

500

In [73]:
myhref[0:10]

['/erling-haaland/profil/spieler/418560',
 '/kylian-mbappe/profil/spieler/342229',
 '/bukayo-saka/profil/spieler/433177',
 '/phil-foden/profil/spieler/406635',
 '/harry-kane/profil/spieler/132098',
 '/rafael-leao/profil/spieler/357164',
 '/khvicha-kvaratskhelia/profil/spieler/502670',
 '/randal-kolo-muani/profil/spieler/487969',
 '/marcus-rashford/profil/spieler/258923',
 '/jack-grealish/profil/spieler/203460']

In [300]:
for i,a in enumerate(myhref):
    myhref[i]="https://www.transfermarkt.us"+a
myhref[0:10]

['https://www.transfermarkt.us/erling-haaland/profil/spieler/418560',
 'https://www.transfermarkt.us/kylian-mbappe/profil/spieler/342229',
 'https://www.transfermarkt.us/bukayo-saka/profil/spieler/433177',
 'https://www.transfermarkt.us/phil-foden/profil/spieler/406635',
 'https://www.transfermarkt.us/harry-kane/profil/spieler/132098',
 'https://www.transfermarkt.us/rafael-leao/profil/spieler/357164',
 'https://www.transfermarkt.us/khvicha-kvaratskhelia/profil/spieler/502670',
 'https://www.transfermarkt.us/randal-kolo-muani/profil/spieler/487969',
 'https://www.transfermarkt.us/marcus-rashford/profil/spieler/258923',
 'https://www.transfermarkt.us/jack-grealish/profil/spieler/203460']

In [49]:
Mylist=[]

In [77]:
Mylist = []  
num_retries = 3  
retry_delay = 5  
user_agent = 'Your User-Agent'  

for url in tqdm(myhref):
    for _ in range(num_retries):
        try:
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser")

            Name = Soup.find("h1", {"class": "data-header__headline-wrapper"}).text.strip()
            SN=Soup.find_all("span",{"class":"data-header__shirt-number"})[0].text 

            All = Soup.find("div", {"class": "data-header__details"})
            Age_elements = All.find_all("span", {"class": "data-header__content"})
            Age = Age_elements[0].text.strip() 
            Position = Age_elements[4].text.strip() 
            Nation = Age_elements[2].text.strip() 
            Height = Age_elements[3].text.strip()

            mydict = {
                    "Name": Name,
                    "Shirt Number": SN,
                    "Age": Age,
                    "Sub-Position": Position,
                    "Country": Nation,
                    "Height": Height
            }
            Mylist.append(mydict)
            break  

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except IndexError as e:
            print(f"Index error in data extraction: {e}")
            break  # Exit the loop if data structure is not as expected
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

100%|█████████████████████████████████████████| 500/500 [07:27<00:00,  1.12it/s]


In [81]:
Firstdf=pd.DataFrame(Mylist)

In [97]:
Cuplist=range(0,500)
Cuplist=list(Cuplist)
for i,a in enumerate(myhref):
    Cuplist[i]=myhref[i].replace("profil","erfolge")

In [299]:
Cuplist[0:10]

['https://www.transfermarkt.us/erling-haaland/erfolge/spieler/418560',
 'https://www.transfermarkt.us/kylian-mbappe/erfolge/spieler/342229',
 'https://www.transfermarkt.us/bukayo-saka/erfolge/spieler/433177',
 'https://www.transfermarkt.us/phil-foden/erfolge/spieler/406635',
 'https://www.transfermarkt.us/harry-kane/erfolge/spieler/132098',
 'https://www.transfermarkt.us/rafael-leao/erfolge/spieler/357164',
 'https://www.transfermarkt.us/khvicha-kvaratskhelia/erfolge/spieler/502670',
 'https://www.transfermarkt.us/randal-kolo-muani/erfolge/spieler/487969',
 'https://www.transfermarkt.us/marcus-rashford/erfolge/spieler/258923',
 'https://www.transfermarkt.us/jack-grealish/erfolge/spieler/203460']

In [101]:
Ownlist=[]

In [102]:
num_retries = 3  
retry_delay = 5  
user_agent = 'Your User-Agent'  

for url in tqdm(Cuplist):
    for _ in range(num_retries):
        try:
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser")
            All = Soup.find("div", {"class": "large-8 columns"})
            Heads=All.find_all("h2",{"class":"content-box-headline"})
            Cups=[]
            for i,a in enumerate(Heads):
                Cups.append(Heads[i].text.strip())
            mydict = {"Cups":Cups}
            Ownlist.append(mydict)
            break  

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except IndexError as e:
            print(f"Index error in data extraction: {e}")
            break  # Exit the loop if data structure is not as expected
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

100%|█████████████████████████████████████████| 500/500 [06:05<00:00,  1.37it/s]


In [105]:
Seconddf=pd.DataFrame(Ownlist)
Seconddf

,Cups
0,"[1x UEFA Best Player in Europe, 5x Footballer ..."
1,"[3x Footballer of the Year, 10x Top goal score..."
2,"[1x English FA Cup winner, 2x English Super Cu..."
3,"[1x Champions League winner, 5x English Champi..."
4,"[6x Top goal scorer, 4x Player of the Year, 1x..."
...,...
495,[]
496,[]
497,[]
498,[1x Portuguese cup winner]


In [112]:
Firstdf=pd.concat([Firstdf,Seconddf],axis=1)

In [113]:
Firstdf

,Name,Shirt Number,Age,Sub-Position,Country,Height,Cups
0,#9 \n ...,\n #9,"Jul 21, 2000 (23)",Centre-Forward,Norway,"1,95 m","[1x UEFA Best Player in Europe, 5x Footballer ..."
1,#7 \n ...,\n #7,"Dec 20, 1998 (24)",Left Winger,France,"1,78 m","[3x Footballer of the Year, 10x Top goal score..."
2,#7 \n ...,\n #7,"Sep 5, 2001 (22)",Right Winger,England,"1,78 m","[1x English FA Cup winner, 2x English Super Cu..."
3,#47 \n ...,\n #47,"May 28, 2000 (23)",Left Winger,England,"1,71 m","[1x Champions League winner, 5x English Champi..."
4,#9 \n ...,\n #9,"Jul 28, 1993 (30)",Centre-Forward,England,"1,88 m","[6x Top goal scorer, 4x Player of the Year, 1x..."
...,...,...,...,...,...,...,...
495,#9 \n ...,\n #9,"Sep 19, 2002 (21)",Centre-Forward,Spain,"1,83 m",[]
496,#25 \n ...,\n #25,"Dec 16, 2005 (17)",Left Winger,Slovakia,"1,84 m",[]
497,#27 \n ...,\n #27,"Apr 16, 2004 (19)",Right Winger,Spain,"1,75 m",[]
498,#21 \n ...,\n #21,"Jul 7, 2003 (20)",Right Winger,Portugal,"1,75 m",[1x Portuguese cup winner]


In [115]:
Withcups=Firstdf.copy()

In [116]:
Statlist=range(0,500)
Statlist=list(Statlist)
for i,a in enumerate(myhref):
    Statlist[i]=myhref[i].replace("profil","leistungsdatendetails")
Statlist[10:20]

['https://www.transfermarkt.us/leroy-sane/leistungsdatendetails/spieler/192565',
 'https://www.transfermarkt.us/xavi-simons/leistungsdatendetails/spieler/566931',
 'https://www.transfermarkt.us/dusan-vlahovic/leistungsdatendetails/spieler/357498',
 'https://www.transfermarkt.us/alexander-isak/leistungsdatendetails/spieler/349066',
 'https://www.transfermarkt.us/evan-ferguson/leistungsdatendetails/spieler/648046',
 'https://www.transfermarkt.us/rasmus-hojlund/leistungsdatendetails/spieler/610442',
 'https://www.transfermarkt.us/kingsley-coman/leistungsdatendetails/spieler/243714',
 'https://www.transfermarkt.us/jeremy-doku/leistungsdatendetails/spieler/486049',
 'https://www.transfermarkt.us/ousmane-dembele/leistungsdatendetails/spieler/288230',
 'https://www.transfermarkt.us/cody-gakpo/leistungsdatendetails/spieler/434675']

In [121]:
R = requests.get("https://www.transfermarkt.us/xavi-simons/leistungsdatendetails/spieler/566931", headers={'User-Agent': 'Your User-Agent'})
Soup = BeautifulSoup(R.text, "html.parser")
All = Soup.find_all("td", {"class": "zentriert"})
Cards=All[4].text
Assists=All[3].text
Goals=All[2].text
Matches=All[1].text
Minutes=Soup.find_all("td",{"class":"rechts"})
Minutes2=Minutes[1].text
print(All)

27.555'


In [129]:
Ownlist2=[]

In [131]:
num_retries = 3  
retry_delay = 5  
user_agent = 'Your User-Agent'  

for url in tqdm(Statlist):
    for _ in range(num_retries):
        try:
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser") 

            All = Soup.find_all("td", {"class": "zentriert"})
            Cards=All[4].text
            Assists=All[3].text
            Goals=All[2].text
            Matches=All[1].text
            Minutes=Soup.find_all("td",{"class":"rechts"})
            Minutes2=Minutes[1].text
            mydict = {"Matches":Matches,
                     "Goals":Goals,
                     "Assists":Assists,
                      "Cards":Cards,
                      "Minutes":Minutes2
                     }
            Ownlist2.append(mydict)
            break  

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except IndexError as e:
            print(f"Index error in data extraction: {e}")
            break  # Exit the loop if data structure is not as expected
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

100%|█████████████████████████████████████████| 500/500 [15:17<00:00,  1.84s/it]


In [132]:
thirddf=pd.DataFrame(Ownlist2)
thirddf

,Matches,Goals,Assists,Cards,Minutes
0,253,202,48,23 / - / -,17.474'
1,339,256,117,43 / - / 3,25.615'
2,248,66,64,24 / - / -,18.266'
3,276,86,58,8 / 1 / -,17.155'
4,529,326,87,48 / 1 / -,40.963'
...,...,...,...,...,...
495,80,23,4,11 / - / -,4.148'
496,47,7,6,1 / - / -,2.907'
497,66,5,3,7 / 1 / 2,3.588'
498,80,14,3,4 / - / -,4.442'


In [133]:
Firstdf=pd.concat([Firstdf,thirddf],axis=1)
Firstdf.head()

,Name,Shirt Number,Age,Sub-Position,Country,Height,Cups,Matches,Goals,Assists,Cards,Minutes
0,#9 \n ...,\n #9,"Jul 21, 2000 (23)",Centre-Forward,Norway,"1,95 m","[1x UEFA Best Player in Europe, 5x Footballer ...",253,202,48,23 / - / -,17.474'
1,#7 \n ...,\n #7,"Dec 20, 1998 (24)",Left Winger,France,"1,78 m","[3x Footballer of the Year, 10x Top goal score...",339,256,117,43 / - / 3,25.615'
2,#7 \n ...,\n #7,"Sep 5, 2001 (22)",Right Winger,England,"1,78 m","[1x English FA Cup winner, 2x English Super Cu...",248,66,64,24 / - / -,18.266'
3,#47 \n ...,\n #47,"May 28, 2000 (23)",Left Winger,England,"1,71 m","[1x Champions League winner, 5x English Champi...",276,86,58,8 / 1 / -,17.155'
4,#9 \n ...,\n #9,"Jul 28, 1993 (30)",Centre-Forward,England,"1,88 m","[6x Top goal scorer, 4x Player of the Year, 1x...",529,326,87,48 / 1 / -,40.963'


In [134]:
InjuryL=range(0,500)
InjuryL=list(InjuryL)
for i,a in enumerate(myhref):
    InjuryL[i]=myhref[i].replace("profil","verletzungen")
InjuryL[10:20]

['https://www.transfermarkt.us/leroy-sane/verletzungen/spieler/192565',
 'https://www.transfermarkt.us/xavi-simons/verletzungen/spieler/566931',
 'https://www.transfermarkt.us/dusan-vlahovic/verletzungen/spieler/357498',
 'https://www.transfermarkt.us/alexander-isak/verletzungen/spieler/349066',
 'https://www.transfermarkt.us/evan-ferguson/verletzungen/spieler/648046',
 'https://www.transfermarkt.us/rasmus-hojlund/verletzungen/spieler/610442',
 'https://www.transfermarkt.us/kingsley-coman/verletzungen/spieler/243714',
 'https://www.transfermarkt.us/jeremy-doku/verletzungen/spieler/486049',
 'https://www.transfermarkt.us/ousmane-dembele/verletzungen/spieler/288230',
 'https://www.transfermarkt.us/cody-gakpo/verletzungen/spieler/434675']

In [157]:
Ownlist3=[]

In [150]:
R = requests.get("https://www.transfermarkt.us/marcus-rashford/verletzungen/spieler/258923", headers={'User-Agent': 'Your User-Agent'})
Soup = BeautifulSoup(R.text, "html.parser")
Value=Soup.find("div",{"class":"data-header__box--small"}).text
            #Inj=[]
Injuries=Soup.find_all("td",{"class":"rechts hauptlink wappen_verletzung"})
print(Value)


€75.00m Last update: Oct 9, 2023



In [158]:
num_retries = 3  
retry_delay = 5  
user_agent = 'Your User-Agent'  

for url in tqdm(InjuryL):
    for _ in range(num_retries):
        try:
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser") 
            Value=Soup.find("div",{"class":"data-header__box--small"}).text
            Inj=[]
            Injuries=Soup.find_all("td",{"class":"rechts hauptlink wappen_verletzung"})
            for a in Injuries:
                Inj.append(a.text)
            mydict = {"Injuries":Inj,
                     "Value":Value,
                     }
            Ownlist3.append(mydict)
            break  

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except IndexError as e:
            print(f"Index error in data extraction: {e}")
            break  # Exit the loop if data structure is not as expected
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

100%|█████████████████████████████████████████| 500/500 [05:07<00:00,  1.62it/s]


In [160]:
fourthdf=pd.DataFrame(Ownlist3)
fourthdf

,Injuries,Value
0,"[3, 2, -, 6, 9, 5, 1, 1, 7, 1, 3, 1, 3, 3, 2]","\n€180.00m Last update: Jun 20, 2023\n"
1,"[3, 1, 1, 2, 1, 1, 4, 4, 1, 1, 1, 3, 7, 1, 1]","\n€180.00m Last update: Jun 27, 2023\n"
2,"[1, 1, 1, -, 1, 2]","\n€120.00m Last update: Jun 20, 2023\n"
3,"[4, 7, 3, 12]","\n€110.00m Last update: Jun 20, 2023\n"
4,"[2, 2, 1, 1, 14, 1, 9, 8, -, 4, 2, 2, 5, 13, 16]","\n€110.00m Last update: Oct 18, 2023\n"
...,...,...
495,[],"\n€3.00m Last update: Oct 13, 2023\n"
496,[],"\n€3.00m Last update: Oct 17, 2023\n"
497,"[1, 3, -]","\n€3.00m Last update: Jun 30, 2023\n"
498,[],"\n€3.00m Last update: Jun 16, 2023\n"


In [161]:
Firstdf=pd.concat([Firstdf,fourthdf],axis=1)
Firstdf.head()

,Name,Shirt Number,Age,Sub-Position,Country,Height,Cups,Matches,Goals,Assists,Cards,Minutes,Injuries,Value
0,#9 \n ...,\n #9,"Jul 21, 2000 (23)",Centre-Forward,Norway,"1,95 m","[1x UEFA Best Player in Europe, 5x Footballer ...",253,202,48,23 / - / -,17.474',"[3, 2, -, 6, 9, 5, 1, 1, 7, 1, 3, 1, 3, 3, 2]","\n€180.00m Last update: Jun 20, 2023\n"
1,#7 \n ...,\n #7,"Dec 20, 1998 (24)",Left Winger,France,"1,78 m","[3x Footballer of the Year, 10x Top goal score...",339,256,117,43 / - / 3,25.615',"[3, 1, 1, 2, 1, 1, 4, 4, 1, 1, 1, 3, 7, 1, 1]","\n€180.00m Last update: Jun 27, 2023\n"
2,#7 \n ...,\n #7,"Sep 5, 2001 (22)",Right Winger,England,"1,78 m","[1x English FA Cup winner, 2x English Super Cu...",248,66,64,24 / - / -,18.266',"[1, 1, 1, -, 1, 2]","\n€120.00m Last update: Jun 20, 2023\n"
3,#47 \n ...,\n #47,"May 28, 2000 (23)",Left Winger,England,"1,71 m","[1x Champions League winner, 5x English Champi...",276,86,58,8 / 1 / -,17.155',"[4, 7, 3, 12]","\n€110.00m Last update: Jun 20, 2023\n"
4,#9 \n ...,\n #9,"Jul 28, 1993 (30)",Centre-Forward,England,"1,88 m","[6x Top goal scorer, 4x Player of the Year, 1x...",529,326,87,48 / 1 / -,40.963',"[2, 2, 1, 1, 14, 1, 9, 8, -, 4, 2, 2, 5, 13, 16]","\n€110.00m Last update: Oct 18, 2023\n"


In [164]:
Final=Firstdf.copy()

In [165]:
Firstdf["Name"] = Firstdf["Name"].apply(lambda x: x.split("\n")[1] if "\n" in x else x)
Firstdf["Name"]

0                                       Erling Haaland
1                                        Kylian Mbappé
2                                          Bukayo Saka
3                                           Phil Foden
4                                           Harry Kane
                            ...                       
495                                    Jon Karrikaburu
496                                          Leo Sauer
497                                     Ilias Akhomach
498                                      Rodrigo Gomes
499                                     Jacob Ondrejka
Name: Name, Length: 500, dtype: object

In [202]:
Firstdf["Links"]=myhref
Firstdf

,Name,Shirt Number,Age,Sub-Position,Country,Height,Cups,Matches,Goals,Assists,Cards,Minutes,Injuries,Value,Links
0,Erling Haaland,\n #9,"Jul 21, 2000 (23)",Centre-Forward,Norway,"1,95 m","[1x UEFA Best Player in Europe, 5x Footballer ...",253,202,48,23 / - / -,17.474',"[3, 2, -, 6, 9, 5, 1, 1, 7, 1, 3, 1, 3, 3, 2]","\n€180.00m Last update: Jun 20, 2023\n",https://www.transfermarkt.us/erling-haaland/pr...
1,Kylian Mbappé,\n #7,"Dec 20, 1998 (24)",Left Winger,France,"1,78 m","[3x Footballer of the Year, 10x Top goal score...",339,256,117,43 / - / 3,25.615',"[3, 1, 1, 2, 1, 1, 4, 4, 1, 1, 1, 3, 7, 1, 1]","\n€180.00m Last update: Jun 27, 2023\n",https://www.transfermarkt.us/kylian-mbappe/pro...
2,Bukayo Saka,\n #7,"Sep 5, 2001 (22)",Right Winger,England,"1,78 m","[1x English FA Cup winner, 2x English Super Cu...",248,66,64,24 / - / -,18.266',"[1, 1, 1, -, 1, 2]","\n€120.00m Last update: Jun 20, 2023\n",https://www.transfermarkt.us/bukayo-saka/profi...
3,Phil Foden,\n #47,"May 28, 2000 (23)",Left Winger,England,"1,71 m","[1x Champions League winner, 5x English Champi...",276,86,58,8 / 1 / -,17.155',"[4, 7, 3, 12]","\n€110.00m Last update: Jun 20, 2023\n",https://www.transfermarkt.us/phil-foden/profil...
4,Harry Kane,\n #9,"Jul 28, 1993 (30)",Centre-Forward,England,"1,88 m","[6x Top goal scorer, 4x Player of the Year, 1x...",529,326,87,48 / 1 / -,40.963',"[2, 2, 1, 1, 14, 1, 9, 8, -, 4, 2, 2, 5, 13, 16]","\n€110.00m Last update: Oct 18, 2023\n",https://www.transfermarkt.us/harry-kane/profil...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Jon Karrikaburu,\n #9,"Sep 19, 2002 (21)",Centre-Forward,Spain,"1,83 m",[],80,23,4,11 / - / -,4.148',[],"\n€3.00m Last update: Oct 13, 2023\n",https://www.transfermarkt.us/jon-karrikaburu/p...
496,Leo Sauer,\n #25,"Dec 16, 2005 (17)",Left Winger,Slovakia,"1,84 m",[],47,7,6,1 / - / -,2.907',[],"\n€3.00m Last update: Oct 17, 2023\n",https://www.transfermarkt.us/leo-sauer/profil/...
497,Ilias Akhomach,\n #27,"Apr 16, 2004 (19)",Right Winger,Spain,"1,75 m",[],66,5,3,7 / 1 / 2,3.588',"[1, 3, -]","\n€3.00m Last update: Jun 30, 2023\n",https://www.transfermarkt.us/ilias-akhomach/pr...
498,Rodrigo Gomes,\n #21,"Jul 7, 2003 (20)",Right Winger,Portugal,"1,75 m",[1x Portuguese cup winner],80,14,3,4 / - / -,4.442',[],"\n€3.00m Last update: Jun 16, 2023\n",https://www.transfermarkt.us/rodrigo-gomes/pro...


##  Scraping 500 most progresed players

In [191]:
myhref2 = []
num_pages = 20  
retry_delay = 5  

for page_num in tqdm(range(1, num_pages + 1)): 
    for _ in range(3):  
        try:
            url = f"https://www.transfermarkt.us/spieler-statistik/marktwertspruenge/marktwertetop/plus/datum/2021-07-01/ausrichtung/Sturm/spielerposition_id//altersklasse/alle/land_id/0/yt0/Show/0//page/{page_num}"
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser")

            Link1 = Soup.find("div", {'class': "responsive-table"})
            if Link1:  
                links = Link1.find_all('a')
                for a in links:
                    href = a.get('href')
                    if href and "profil/spieler/" in href:
                        myhref2.append(href)
                break 
        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)

100%|███████████████████████████████████████████| 20/20 [00:45<00:00,  2.28s/it]


In [192]:
len(myhref2)

500

In [298]:
for i,a in enumerate(myhref2):
    myhref2[i]="https://www.transfermarkt.us"+a
myhref2[0:10]

['https://www.transfermarkt.us/vinicius-junior/profil/spieler/371998',
 'https://www.transfermarkt.us/randal-kolo-muani/profil/spieler/487969',
 'https://www.transfermarkt.us/victor-osimhen/profil/spieler/401923',
 'https://www.transfermarkt.us/xavi-simons/profil/spieler/566931',
 'https://www.transfermarkt.us/khvicha-kvaratskhelia/profil/spieler/502670',
 'https://www.transfermarkt.us/rodrygo/profil/spieler/412363',
 'https://www.transfermarkt.us/rafael-leao/profil/spieler/357164',
 'https://www.transfermarkt.us/julian-alvarez/profil/spieler/576024',
 'https://www.transfermarkt.us/evan-ferguson/profil/spieler/648046',
 'https://www.transfermarkt.us/rasmus-hojlund/profil/spieler/610442']

In [194]:
Mylist2 = []  
num_retries = 3  
retry_delay = 5  
user_agent = 'Your User-Agent'  

for url in tqdm(myhref2):
    for _ in range(num_retries):
        try:
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser")

            try:
                Name = Soup.find("h1", {"class": "data-header__headline-wrapper"}).text.strip()
            except AttributeError:
                Name = ""

            try:
                SN = Soup.find_all("span", {"class": "data-header__shirt-number"})[0].text
            except IndexError:
                SN = 0
 
            
            try:
                All = Soup.find("div", {"class": "data-header__details"})
                Age_elements = All.find_all("span", {"class": "data-header__content"})
                Age = Age_elements[0].text.strip()
                Position = Age_elements[4].text.strip()
                Nation = Age_elements[2].text.strip()
                Height = Age_elements[3].text.strip()
            except (IndexError, AttributeError):
                Age = Position = Nation = Height = 0

            mydict = {
                    "Name": Name,
                    "Shirt Number": SN,
                    "Age": Age,
                    "Sub-Position": Position,
                    "Country": Nation,
                    "Height": Height
            }
            Mylist2.append(mydict)
            break  

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

100%|█████████████████████████████████████████| 500/500 [04:47<00:00,  1.74it/s]


In [195]:
Valuedf=pd.DataFrame(Mylist2)
Valuedf

,Name,Shirt Number,Age,Sub-Position,Country,Height
0,#7 \nVinicius Junior,\n #7,"Jul 12, 2000 (23)",Left Winger,Brazil,"1,76 m"
1,#23 \n ...,\n #23,"Dec 5, 1998 (24)",Centre-Forward,France,"1,87 m"
2,#9 \n ...,\n #9,"Dec 29, 1998 (24)",Centre-Forward,Nigeria,"1,86 m"
3,#20 \n ...,\n #20,"Apr 21, 2003 (20)",Right Winger,Netherlands,"1,79 m"
4,#77 \n ...,\n #77,"Feb 12, 2001 (22)",Left Winger,Georgia,"1,83 m"
...,...,...,...,...,...,...
495,#21 \n ...,\n #21,"Jul 9, 1997 (26)",Centre-Forward,Germany,"1,90 m"
496,#14 \n ...,\n #14,"Feb 21, 1998 (25)",Centre-Forward,Cote d'Ivoire,"1,89 m"
497,#28 \n ...,\n #28,"Feb 17, 1996 (27)",Centre-Forward,Colombia,"1,88 m"
498,#24 \n ...,\n #24,"Jan 25, 2001 (22)",Centre-Forward,Italy,"1,89 m"


In [209]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time

DiffList = []
num_pages = 20
retry_delay = 5

for page_num in tqdm(range(1, num_pages + 1)): 
    page_processed = False
    for _ in range(3): 
        if page_processed:
            break  
        try:
            url = f"https://www.transfermarkt.us/spieler-statistik/marktwertspruenge/marktwertetop/plus/datum/2021-07-01/ausrichtung/Sturm/spielerposition_id//altersklasse/alle/land_id/0/yt0/Show/0//page/{page_num}"
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser")
            Pageo = 0
            for i in range(25):
                DiffPercent = Soup.find_all("td", {"class": "rechts greentext"})[Pageo].text
                Pageo += 1
                DifferenceValue = Soup.find_all("td", {"class": "rechts greentext"})[Pageo].text
                Pageo += 1
                mydict = {"DiffPercent": DiffPercent, "DifferenceValue": DifferenceValue}
                DiffList.append(mydict)
            page_processed = True  

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except IndexError as e:
            print(f"Index error in data extraction: {e}")
            break  
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

100%|███████████████████████████████████████████| 20/20 [00:56<00:00,  2.81s/it]


In [211]:
Diffpd=pd.DataFrame(DiffList)
Diffpd

,DiffPercent,DifferenceValue
0,275.0 %,+€110.00m
1,788.9 %,+€71.00m
2,140.0 %,+€70.00m
3,13900.0 %,+€69.50m
4,372.2 %,+€67.00m
...,...,...
495,650.0 %,+€2.60m
496,650.0 %,+€2.60m
497,566.7 %,+€2.55m
498,1020.0 %,+€2.55m


In [216]:
Valuedf=pd.concat([Valuedf,Diffpd],axis=1)
Valuedf

,Name,Shirt Number,Age,Sub-Position,Country,Height,DiffPercent,DifferenceValue
0,#7 \nVinicius Junior,\n #7,"Jul 12, 2000 (23)",Left Winger,Brazil,"1,76 m",275.0 %,+€110.00m
1,#23 \n ...,\n #23,"Dec 5, 1998 (24)",Centre-Forward,France,"1,87 m",788.9 %,+€71.00m
2,#9 \n ...,\n #9,"Dec 29, 1998 (24)",Centre-Forward,Nigeria,"1,86 m",140.0 %,+€70.00m
3,#20 \n ...,\n #20,"Apr 21, 2003 (20)",Right Winger,Netherlands,"1,79 m",13900.0 %,+€69.50m
4,#77 \n ...,\n #77,"Feb 12, 2001 (22)",Left Winger,Georgia,"1,83 m",372.2 %,+€67.00m
...,...,...,...,...,...,...,...,...
495,#21 \n ...,\n #21,"Jul 9, 1997 (26)",Centre-Forward,Germany,"1,90 m",650.0 %,+€2.60m
496,#14 \n ...,\n #14,"Feb 21, 1998 (25)",Centre-Forward,Cote d'Ivoire,"1,89 m",650.0 %,+€2.60m
497,#28 \n ...,\n #28,"Feb 17, 1996 (27)",Centre-Forward,Colombia,"1,88 m",566.7 %,+€2.55m
498,#24 \n ...,\n #24,"Jan 25, 2001 (22)",Centre-Forward,Italy,"1,89 m",1020.0 %,+€2.55m


In [217]:
Valuedf["Name"] = Valuedf["Name"].apply(lambda x: x.split("\n")[1] if "\n" in x else x)
Valuedf

,Name,Shirt Number,Age,Sub-Position,Country,Height,DiffPercent,DifferenceValue
0,Vinicius Junior,\n #7,"Jul 12, 2000 (23)",Left Winger,Brazil,"1,76 m",275.0 %,+€110.00m
1,Randal Kolo Muani,\n #23,"Dec 5, 1998 (24)",Centre-Forward,France,"1,87 m",788.9 %,+€71.00m
2,Victor Osimhen,\n #9,"Dec 29, 1998 (24)",Centre-Forward,Nigeria,"1,86 m",140.0 %,+€70.00m
3,Xavi Simons,\n #20,"Apr 21, 2003 (20)",Right Winger,Netherlands,"1,79 m",13900.0 %,+€69.50m
4,Khvicha Kvarat...,\n #77,"Feb 12, 2001 (22)",Left Winger,Georgia,"1,83 m",372.2 %,+€67.00m
...,...,...,...,...,...,...,...,...
495,Phillip Tietz,\n #21,"Jul 9, 1997 (26)",Centre-Forward,Germany,"1,90 m",650.0 %,+€2.60m
496,Wilfried Kanga,\n #14,"Feb 21, 1998 (25)",Centre-Forward,Cote d'Ivoire,"1,89 m",650.0 %,+€2.60m
497,Diber Cambindo,\n #28,"Feb 17, 1996 (27)",Centre-Forward,Colombia,"1,88 m",566.7 %,+€2.55m
498,Luca Moro,\n #24,"Jan 25, 2001 (22)",Centre-Forward,Italy,"1,89 m",1020.0 %,+€2.55m


In [220]:
Cuplist2=range(0,500)
Cuplist2=list(Cuplist2)
for i,a in enumerate(myhref2):
    Cuplist2[i]=myhref2[i].replace("profil","erfolge")

In [297]:
Cuplist2[0:10]

['https://www.transfermarkt.us/vinicius-junior/erfolge/spieler/371998',
 'https://www.transfermarkt.us/randal-kolo-muani/erfolge/spieler/487969',
 'https://www.transfermarkt.us/victor-osimhen/erfolge/spieler/401923',
 'https://www.transfermarkt.us/xavi-simons/erfolge/spieler/566931',
 'https://www.transfermarkt.us/khvicha-kvaratskhelia/erfolge/spieler/502670',
 'https://www.transfermarkt.us/rodrygo/erfolge/spieler/412363',
 'https://www.transfermarkt.us/rafael-leao/erfolge/spieler/357164',
 'https://www.transfermarkt.us/julian-alvarez/erfolge/spieler/576024',
 'https://www.transfermarkt.us/evan-ferguson/erfolge/spieler/648046',
 'https://www.transfermarkt.us/rasmus-hojlund/erfolge/spieler/610442']

In [222]:
Ownlist2=[]

In [223]:
num_retries = 3  
retry_delay = 5  
user_agent = 'Your User-Agent'  

for url in tqdm(Cuplist2):
    for _ in range(num_retries):
        try:
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser")
            All = Soup.find("div", {"class": "large-8 columns"})
            Heads=All.find_all("h2",{"class":"content-box-headline"})
            Cups=[]
            for i,a in enumerate(Heads):
                Cups.append(Heads[i].text.strip())
            mydict = {"Cups":Cups}
            Ownlist2.append(mydict)
            break  

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except IndexError as e:
            print(f"Index error in data extraction: {e}")
            break  # Exit the loop if data structure is not as expected
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

100%|█████████████████████████████████████████| 500/500 [04:30<00:00,  1.85it/s]


In [224]:
Seconddf2=pd.DataFrame(Ownlist2)
Seconddf2

,Cups
0,"[1x Top goal scorer, 1x Champions League winne..."
1,"[1x Top goal scorer, 1x French cup winner]"
2,"[3x Top goal scorer, 1x TM-Player of the seaso..."
3,"[1x Top goal scorer, 1x TM-Player of the seaso..."
4,"[3x Footballer of the Year, 1x Top goal scorer..."
...,...
495,"[1x Westphalia Cup winner, 1x Hessen Cup winner]"
496,[]
497,[1x Top goal scorer]
498,[1x Italienischer Zweitligameister]


In [225]:
Valuedf=pd.concat([Valuedf,Seconddf2],axis=1)
Valuedf

,Name,Shirt Number,Age,Sub-Position,Country,Height,DiffPercent,DifferenceValue,Cups
0,Vinicius Junior,\n #7,"Jul 12, 2000 (23)",Left Winger,Brazil,"1,76 m",275.0 %,+€110.00m,"[1x Top goal scorer, 1x Champions League winne..."
1,Randal Kolo Muani,\n #23,"Dec 5, 1998 (24)",Centre-Forward,France,"1,87 m",788.9 %,+€71.00m,"[1x Top goal scorer, 1x French cup winner]"
2,Victor Osimhen,\n #9,"Dec 29, 1998 (24)",Centre-Forward,Nigeria,"1,86 m",140.0 %,+€70.00m,"[3x Top goal scorer, 1x TM-Player of the seaso..."
3,Xavi Simons,\n #20,"Apr 21, 2003 (20)",Right Winger,Netherlands,"1,79 m",13900.0 %,+€69.50m,"[1x Top goal scorer, 1x TM-Player of the seaso..."
4,Khvicha Kvarat...,\n #77,"Feb 12, 2001 (22)",Left Winger,Georgia,"1,83 m",372.2 %,+€67.00m,"[3x Footballer of the Year, 1x Top goal scorer..."
...,...,...,...,...,...,...,...,...,...
495,Phillip Tietz,\n #21,"Jul 9, 1997 (26)",Centre-Forward,Germany,"1,90 m",650.0 %,+€2.60m,"[1x Westphalia Cup winner, 1x Hessen Cup winner]"
496,Wilfried Kanga,\n #14,"Feb 21, 1998 (25)",Centre-Forward,Cote d'Ivoire,"1,89 m",650.0 %,+€2.60m,[]
497,Diber Cambindo,\n #28,"Feb 17, 1996 (27)",Centre-Forward,Colombia,"1,88 m",566.7 %,+€2.55m,[1x Top goal scorer]
498,Luca Moro,\n #24,"Jan 25, 2001 (22)",Centre-Forward,Italy,"1,89 m",1020.0 %,+€2.55m,[1x Italienischer Zweitligameister]


In [227]:
Statlist2=range(0,500)
Statlist2=list(Statlist2)
for i,a in enumerate(myhref2):
    Statlist2[i]=myhref2[i].replace("profil","leistungsdatendetails")

In [230]:
Ownlist2=[]

In [231]:
num_retries = 3  
retry_delay = 5  
user_agent = 'Your User-Agent'  

for url in tqdm(Statlist2):
    for _ in range(num_retries):
        try:
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser") 

            All = Soup.find_all("td", {"class": "zentriert"})
            Cards=All[4].text
            Assists=All[3].text
            Goals=All[2].text
            Matches=All[1].text
            Minutes=Soup.find_all("td",{"class":"rechts"})
            Minutes2=Minutes[1].text
            mydict = {"Matches":Matches,
                     "Goals":Goals,
                     "Assists":Assists,
                      "Cards":Cards,
                      "Minutes":Minutes2
                     }
            Ownlist2.append(mydict)
            break  

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except IndexError as e:
            print(f"Index error in data extraction: {e}")
            break  
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

100%|█████████████████████████████████████████| 500/500 [09:32<00:00,  1.15s/it]


In [232]:
Stats=pd.DataFrame(Ownlist2)
Stats

,Matches,Goals,Assists,Cards,Minutes
0,291,77,73,46 / 1 / 1,18.878'
1,202,68,44,25 / 2 / 2,15.218'
2,201,103,25,24 / 1 / 1,14.282'
3,90,33,26,14 / - / -,6.498'
4,174,39,46,23 / 1 / -,11.796'
...,...,...,...,...,...
495,286,93,50,22 / - / 1,19.366'
496,161,39,8,14 / - / -,8.161'
497,161,48,7,28 / - / 1,8.733'
498,128,34,4,11 / - / -,8.030'


In [233]:
Valuedf=pd.concat([Valuedf,Stats],axis=1)
Valuedf

,Name,Shirt Number,Age,Sub-Position,Country,Height,DiffPercent,DifferenceValue,Cups,Matches,Goals,Assists,Cards,Minutes
0,Vinicius Junior,\n #7,"Jul 12, 2000 (23)",Left Winger,Brazil,"1,76 m",275.0 %,+€110.00m,"[1x Top goal scorer, 1x Champions League winne...",291,77,73,46 / 1 / 1,18.878'
1,Randal Kolo Muani,\n #23,"Dec 5, 1998 (24)",Centre-Forward,France,"1,87 m",788.9 %,+€71.00m,"[1x Top goal scorer, 1x French cup winner]",202,68,44,25 / 2 / 2,15.218'
2,Victor Osimhen,\n #9,"Dec 29, 1998 (24)",Centre-Forward,Nigeria,"1,86 m",140.0 %,+€70.00m,"[3x Top goal scorer, 1x TM-Player of the seaso...",201,103,25,24 / 1 / 1,14.282'
3,Xavi Simons,\n #20,"Apr 21, 2003 (20)",Right Winger,Netherlands,"1,79 m",13900.0 %,+€69.50m,"[1x Top goal scorer, 1x TM-Player of the seaso...",90,33,26,14 / - / -,6.498'
4,Khvicha Kvarat...,\n #77,"Feb 12, 2001 (22)",Left Winger,Georgia,"1,83 m",372.2 %,+€67.00m,"[3x Footballer of the Year, 1x Top goal scorer...",174,39,46,23 / 1 / -,11.796'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Phillip Tietz,\n #21,"Jul 9, 1997 (26)",Centre-Forward,Germany,"1,90 m",650.0 %,+€2.60m,"[1x Westphalia Cup winner, 1x Hessen Cup winner]",286,93,50,22 / - / 1,19.366'
496,Wilfried Kanga,\n #14,"Feb 21, 1998 (25)",Centre-Forward,Cote d'Ivoire,"1,89 m",650.0 %,+€2.60m,[],161,39,8,14 / - / -,8.161'
497,Diber Cambindo,\n #28,"Feb 17, 1996 (27)",Centre-Forward,Colombia,"1,88 m",566.7 %,+€2.55m,[1x Top goal scorer],161,48,7,28 / - / 1,8.733'
498,Luca Moro,\n #24,"Jan 25, 2001 (22)",Centre-Forward,Italy,"1,89 m",1020.0 %,+€2.55m,[1x Italienischer Zweitligameister],128,34,4,11 / - / -,8.030'


In [234]:
InjuryL2=range(0,500)
InjuryL2=list(InjuryL2)
for i,a in enumerate(myhref):
    InjuryL2[i]=myhref2[i].replace("profil","verletzungen")
InjuryL2[10:20]

['https://www.transfermarkt.us/gabriel-martinelli/verletzungen/spieler/655488',
 'https://www.transfermarkt.us/luis-diaz/verletzungen/spieler/480692',
 'https://www.transfermarkt.us/bukayo-saka/verletzungen/spieler/433177',
 'https://www.transfermarkt.us/erling-haaland/verletzungen/spieler/418560',
 'https://www.transfermarkt.us/mathys-tel/verletzungen/spieler/801734',
 'https://www.transfermarkt.us/lamine-yamal/verletzungen/spieler/937958',
 'https://www.transfermarkt.us/kaoru-mitoma/verletzungen/spieler/504849',
 'https://www.transfermarkt.us/brennan-johnson/verletzungen/spieler/470607',
 'https://www.transfermarkt.us/darwin-nunez/verletzungen/spieler/546543',
 'https://www.transfermarkt.us/mykhaylo-mudryk/verletzungen/spieler/537860']

In [235]:
Ownlist2=[]

In [236]:
num_retries = 3  
retry_delay = 5  
user_agent = 'Your User-Agent'  

for url in tqdm(InjuryL2):
    for _ in range(num_retries):
        try:
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser") 
            Value=Soup.find("div",{"class":"data-header__box--small"}).text
            Inj=[]
            Injuries=Soup.find_all("td",{"class":"rechts hauptlink wappen_verletzung"})
            for a in Injuries:
                Inj.append(a.text)
            mydict = {"Injuries":Inj,
                     "Value":Value,
                     }
            Ownlist2.append(mydict)
            break  

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except IndexError as e:
            print(f"Index error in data extraction: {e}")
            break  
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

100%|█████████████████████████████████████████| 500/500 [04:29<00:00,  1.85it/s]


In [238]:
Injuries=pd.DataFrame(Ownlist2)
Injuries

,Injuries,Value
0,"[4, 2, 2, 1, 11]","\n€150.00m Last update: Jun 13, 2023\n"
1,[1],"\n€80.00m Last update: Jun 22, 2023\n"
2,"[-, 4, 7, -, 2, 13, 2, 3, 6, 15, 11, 6, 2, ?]","\n€120.00m Last update: Jun 15, 2023\n"
3,[],"\n€70.00m Last update: Oct 18, 2023\n"
4,"[1, -, -, 2, 3, 3, 7, 5, 2, 5, 1]","\n€85.00m Last update: Jun 15, 2023\n"
...,...,...
495,"[1, -, 16, 1, 1, 1]","\n€3.00m Last update: Oct 18, 2023\n"
496,"[5, 1, 1, 5, 3, 3, 2, 2, 7]","\n€3.00m Last update: Jun 22, 2023\n"
497,[],"\n€3.00m Last update: Oct 3, 2023\n"
498,[],"\n€2.80m Last update: Jun 12, 2023\n"


In [241]:
Valuedf=pd.concat([Valuedf,Injuries],axis=1)

In [244]:
Valuedf["Links"]=myhref2

In [256]:
Ownlist2=[]

In [258]:
num_retries = 3  
retry_delay = 5  
user_agent = 'Your User-Agent'  

for url in tqdm(myhref):
    for _ in range(num_retries):
        try:
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser") 
            Foot=Soup.find_all("span",{"class":"info-table__content info-table__content--bold"})
            Foot1=Foot[7].text
            Foot2=Foot[6].text
            mydict = {"Foot":Foot1,
                      "Isfoot":Foot2
                     }
            Ownlist2.append(mydict)
            break  

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except IndexError as e:
            print(f"Index error in data extraction: {e}")
            break 
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

100%|█████████████████████████████████████████| 500/500 [07:46<00:00,  1.07it/s]


In [260]:
Footel=pd.DataFrame(Ownlist2)
Footel

,Foot,Isfoot
0,left,\n Attack - Centre-Forward ...
1,right,\n Attack - Left Winger ...
2,"\n Jul 1, 2019 ...",left
3,left,\n Attack - Left Winger ...
4,right,\n Attack - Centre-Forward ...
...,...,...
495,right,\n Attack - Centre-Forward ...
496,"\n Sep 1, 2023 ...",right
497,left,\n Attack - Right Winger ...
498,right,\n Attack - Right Winger ...


In [261]:
Firstdf=pd.concat([Firstdf,Footel],axis=1)

In [265]:
Firstdf["Foot"].value_counts()

right                                                                 252
left                                                                   70
both                                                                   15
\n                            Jul 1, 2022                              15
\n                            Jul 1, 2023                               9
                                                                     ... 
\n                            Sep 2, 2019                               1
\n                            Aug 31, 2018                              1
\n                            Jul 5, 2023                               1
\n                            Jul 25, 2023                              1
\n                            Jan 5, 2016                               1
Name: Foot, Length: 103, dtype: int64

In [ ]:
Ownlist2=[]

In [266]:
num_retries = 3  
retry_delay = 5  
user_agent = 'Your User-Agent'  

for url in tqdm(myhref2):
    for _ in range(num_retries):
        try:
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser") 
            Foot=Soup.find_all("span",{"class":"info-table__content info-table__content--bold"})
            Foot1=Foot[7].text
            Foot2=Foot[6].text
            mydict = {"Foot":Foot1,
                      "Isfoot":Foot2
                     }
            Ownlist2.append(mydict)
            break  

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except IndexError as e:
            print(f"Index error in data extraction: {e}")
            break  # Exit the loop if data structure is not as expected
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

 88%|████████████████████████████████████▏    | 441/500 [06:28<01:35,  1.62s/it]

Request error: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))


100%|█████████████████████████████████████████| 500/500 [07:46<00:00,  1.07it/s]


In [296]:
Footel2=pd.DataFrame(Ownlist2)

In [290]:
Footel2

,Foot,Isfoot
0,right,\n Attack - Left Winger ...
1,"\n Sep 1, 2023 ...",right
2,right,\n Attack - Centre-Forward ...
3,right,\n Attack - Right Winger ...
4,both,\n Attack - Left Winger ...
...,...,...
495,"\n Jul 12, 2023 ...",right
496,right,\n Attack - Centre-Forward ...
497,right,\n Attack - Centre-Forward ...
498,"\n Jul 17, 2023 ...",right


In [291]:
Valuedf=pd.concat([Valuedf,Footel2],axis=1)

In [294]:
Valuedf["Isfoot"].value_counts()

\n                    Attack - Centre-Forward                         168
\n                    Attack - Left Winger                            102
\n                    Attack - Right Winger                            90
right                                                                  83
left                                                                   36
both                                                                    5
\n                    Attack - Second Striker                           4
\n                            Jul 1, 2022                               2
\n                            Jan 5, 2022                               1
\n                            Aug 1, 2022                               1
\n                            Jan 1, 2023                               1
\n                            Jul 6, 2023                               1
\n                            Jul 2, 2023                               1
\n                            Aug 30, 

In [338]:
def parse_and_sum_injuries(injury_list):
    if isinstance(injury_list, str):
        injury_list = injury_list.replace("'", "").strip("[]").split(", ")

    return sum(int(item) if item.isdigit() and item not in ['-', '?'] else 0 for item in injury_list)


In [342]:
Firstdf['Injury_Sum'] = Firstdf['Injuries'].apply(parse_and_sum_injuries)
Firstdf=Firstdf.drop(["Injuries"],axis=1)
Firstdf

,Name,Shirt Number,Age,Sub-Position,Country,Height,Cups,Matches,Goals,Assists,Cards,Minutes,Value,Links,Foot,Isfoot,Injury_Sum
0,Erling Haaland,\n #9,"Jul 21, 2000 (23)",Centre-Forward,Norway,"1,95 m","[1x UEFA Best Player in Europe, 5x Footballer ...",253,202,48,23 / - / -,17.474',"\n€180.00m Last update: Jun 20, 2023\n",https://www.transfermarkt.us/erling-haaland/pr...,left,\n Attack - Centre-Forward ...,47
1,Kylian Mbappé,\n #7,"Dec 20, 1998 (24)",Left Winger,France,"1,78 m","[3x Footballer of the Year, 10x Top goal score...",339,256,117,43 / - / 3,25.615',"\n€180.00m Last update: Jun 27, 2023\n",https://www.transfermarkt.us/kylian-mbappe/pro...,right,\n Attack - Left Winger ...,32
2,Bukayo Saka,\n #7,"Sep 5, 2001 (22)",Right Winger,England,"1,78 m","[1x English FA Cup winner, 2x English Super Cu...",248,66,64,24 / - / -,18.266',"\n€120.00m Last update: Jun 20, 2023\n",https://www.transfermarkt.us/bukayo-saka/profi...,"\n Jul 1, 2019 ...",left,6
3,Phil Foden,\n #47,"May 28, 2000 (23)",Left Winger,England,"1,71 m","[1x Champions League winner, 5x English Champi...",276,86,58,8 / 1 / -,17.155',"\n€110.00m Last update: Jun 20, 2023\n",https://www.transfermarkt.us/phil-foden/profil...,left,\n Attack - Left Winger ...,26
4,Harry Kane,\n #9,"Jul 28, 1993 (30)",Centre-Forward,England,"1,88 m","[6x Top goal scorer, 4x Player of the Year, 1x...",529,326,87,48 / 1 / -,40.963',"\n€110.00m Last update: Oct 18, 2023\n",https://www.transfermarkt.us/harry-kane/profil...,right,\n Attack - Centre-Forward ...,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Jon Karrikaburu,\n #9,"Sep 19, 2002 (21)",Centre-Forward,Spain,"1,83 m",[],80,23,4,11 / - / -,4.148',"\n€3.00m Last update: Oct 13, 2023\n",https://www.transfermarkt.us/jon-karrikaburu/p...,right,\n Attack - Centre-Forward ...,0
496,Leo Sauer,\n #25,"Dec 16, 2005 (17)",Left Winger,Slovakia,"1,84 m",[],47,7,6,1 / - / -,2.907',"\n€3.00m Last update: Oct 17, 2023\n",https://www.transfermarkt.us/leo-sauer/profil/...,"\n Sep 1, 2023 ...",right,0
497,Ilias Akhomach,\n #27,"Apr 16, 2004 (19)",Right Winger,Spain,"1,75 m",[],66,5,3,7 / 1 / 2,3.588',"\n€3.00m Last update: Jun 30, 2023\n",https://www.transfermarkt.us/ilias-akhomach/pr...,left,\n Attack - Right Winger ...,4
498,Rodrigo Gomes,\n #21,"Jul 7, 2003 (20)",Right Winger,Portugal,"1,75 m",[1x Portuguese cup winner],80,14,3,4 / - / -,4.442',"\n€3.00m Last update: Jun 16, 2023\n",https://www.transfermarkt.us/rodrigo-gomes/pro...,right,\n Attack - Right Winger ...,0


In [343]:
Valuedf['Injury_Sum'] = Valuedf['Injuries'].apply(parse_and_sum_injuries)
Valuedf=Valuedf.drop(["Injuries"],axis=1)
Valuedf

,Name,Shirt Number,Age,Sub-Position,Country,Height,DiffPercent,DifferenceValue,Cups,Matches,Goals,Assists,Cards,Minutes,Value,Links,Foot,Isfoot,Injury_Sum
0,Vinicius Junior,\n #7,"Jul 12, 2000 (23)",Left Winger,Brazil,"1,76 m",275.0 %,+€110.00m,"[1x Top goal scorer, 1x Champions League winne...",291,77,73,46 / 1 / 1,18.878',"\n€150.00m Last update: Jun 13, 2023\n",https://www.transfermarkt.us/vinicius-junior/p...,right,\n Attack - Left Winger ...,20
1,Randal Kolo Muani,\n #23,"Dec 5, 1998 (24)",Centre-Forward,France,"1,87 m",788.9 %,+€71.00m,"[1x Top goal scorer, 1x French cup winner]",202,68,44,25 / 2 / 2,15.218',"\n€80.00m Last update: Jun 22, 2023\n",https://www.transfermarkt.us/randal-kolo-muani...,"\n Sep 1, 2023 ...",right,1
2,Victor Osimhen,\n #9,"Dec 29, 1998 (24)",Centre-Forward,Nigeria,"1,86 m",140.0 %,+€70.00m,"[3x Top goal scorer, 1x TM-Player of the seaso...",201,103,25,24 / 1 / 1,14.282',"\n€120.00m Last update: Jun 15, 2023\n",https://www.transfermarkt.us/victor-osimhen/pr...,right,\n Attack - Centre-Forward ...,71
3,Xavi Simons,\n #20,"Apr 21, 2003 (20)",Right Winger,Netherlands,"1,79 m",13900.0 %,+€69.50m,"[1x Top goal scorer, 1x TM-Player of the seaso...",90,33,26,14 / - / -,6.498',"\n€70.00m Last update: Oct 18, 2023\n",https://www.transfermarkt.us/xavi-simons/profi...,right,\n Attack - Right Winger ...,0
4,Khvicha Kvarat...,\n #77,"Feb 12, 2001 (22)",Left Winger,Georgia,"1,83 m",372.2 %,+€67.00m,"[3x Footballer of the Year, 1x Top goal scorer...",174,39,46,23 / 1 / -,11.796',"\n€85.00m Last update: Jun 15, 2023\n",https://www.transfermarkt.us/khvicha-kvaratskh...,both,\n Attack - Left Winger ...,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Phillip Tietz,\n #21,"Jul 9, 1997 (26)",Centre-Forward,Germany,"1,90 m",650.0 %,+€2.60m,"[1x Westphalia Cup winner, 1x Hessen Cup winner]",286,93,50,22 / - / 1,19.366',"\n€3.00m Last update: Oct 18, 2023\n",https://www.transfermarkt.us/phillip-tietz/pro...,"\n Jul 12, 2023 ...",right,20
496,Wilfried Kanga,\n #14,"Feb 21, 1998 (25)",Centre-Forward,Cote d'Ivoire,"1,89 m",650.0 %,+€2.60m,[],161,39,8,14 / - / -,8.161',"\n€3.00m Last update: Jun 22, 2023\n",https://www.transfermarkt.us/wilfried-kanga/pr...,right,\n Attack - Centre-Forward ...,29
497,Diber Cambindo,\n #28,"Feb 17, 1996 (27)",Centre-Forward,Colombia,"1,88 m",566.7 %,+€2.55m,[1x Top goal scorer],161,48,7,28 / - / 1,8.733',"\n€3.00m Last update: Oct 3, 2023\n",https://www.transfermarkt.us/diber-cambindo/pr...,right,\n Attack - Centre-Forward ...,0
498,Luca Moro,\n #24,"Jan 25, 2001 (22)",Centre-Forward,Italy,"1,89 m",1020.0 %,+€2.55m,[1x Italienischer Zweitligameister],128,34,4,11 / - / -,8.030',"\n€2.80m Last update: Jun 12, 2023\n",https://www.transfermarkt.us/luca-moro/profil/...,"\n Jul 17, 2023 ...",right,0


In [347]:
def clean_cups_column(cups_column_value):
    if isinstance(cups_column_value, str):
        cups_list = cups_column_value.strip("[]").replace("'", "").split(", ")
    else:
        cups_list = cups_column_value
    
    return ', '.join(cups_list)

In [352]:
Valuedf['Achievemnts'] = Valuedf['Cups'].apply(clean_cups_column)
Valuedf=Valuedf.drop(["Cups"],axis=1)
Valuedf

,Name,Shirt Number,Age,Sub-Position,Country,Height,DiffPercent,DifferenceValue,Matches,Goals,Assists,Cards,Minutes,Value,Links,Foot,Isfoot,Injury_Sum,Achievemnts
0,Vinicius Junior,\n #7,"Jul 12, 2000 (23)",Left Winger,Brazil,"1,76 m",275.0 %,+€110.00m,291,77,73,46 / 1 / 1,18.878',"\n€150.00m Last update: Jun 13, 2023\n",https://www.transfermarkt.us/vinicius-junior/p...,right,\n Attack - Left Winger ...,20,"1x Top goal scorer, 1x Champions League winner..."
1,Randal Kolo Muani,\n #23,"Dec 5, 1998 (24)",Centre-Forward,France,"1,87 m",788.9 %,+€71.00m,202,68,44,25 / 2 / 2,15.218',"\n€80.00m Last update: Jun 22, 2023\n",https://www.transfermarkt.us/randal-kolo-muani...,"\n Sep 1, 2023 ...",right,1,"1x Top goal scorer, 1x French cup winner"
2,Victor Osimhen,\n #9,"Dec 29, 1998 (24)",Centre-Forward,Nigeria,"1,86 m",140.0 %,+€70.00m,201,103,25,24 / 1 / 1,14.282',"\n€120.00m Last update: Jun 15, 2023\n",https://www.transfermarkt.us/victor-osimhen/pr...,right,\n Attack - Centre-Forward ...,71,"3x Top goal scorer, 1x TM-Player of the season..."
3,Xavi Simons,\n #20,"Apr 21, 2003 (20)",Right Winger,Netherlands,"1,79 m",13900.0 %,+€69.50m,90,33,26,14 / - / -,6.498',"\n€70.00m Last update: Oct 18, 2023\n",https://www.transfermarkt.us/xavi-simons/profi...,right,\n Attack - Right Winger ...,0,"1x Top goal scorer, 1x TM-Player of the season..."
4,Khvicha Kvarat...,\n #77,"Feb 12, 2001 (22)",Left Winger,Georgia,"1,83 m",372.2 %,+€67.00m,174,39,46,23 / 1 / -,11.796',"\n€85.00m Last update: Jun 15, 2023\n",https://www.transfermarkt.us/khvicha-kvaratskh...,both,\n Attack - Left Winger ...,29,"3x Footballer of the Year, 1x Top goal scorer,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Phillip Tietz,\n #21,"Jul 9, 1997 (26)",Centre-Forward,Germany,"1,90 m",650.0 %,+€2.60m,286,93,50,22 / - / 1,19.366',"\n€3.00m Last update: Oct 18, 2023\n",https://www.transfermarkt.us/phillip-tietz/pro...,"\n Jul 12, 2023 ...",right,20,"1x Westphalia Cup winner, 1x Hessen Cup winner"
496,Wilfried Kanga,\n #14,"Feb 21, 1998 (25)",Centre-Forward,Cote d'Ivoire,"1,89 m",650.0 %,+€2.60m,161,39,8,14 / - / -,8.161',"\n€3.00m Last update: Jun 22, 2023\n",https://www.transfermarkt.us/wilfried-kanga/pr...,right,\n Attack - Centre-Forward ...,29,
497,Diber Cambindo,\n #28,"Feb 17, 1996 (27)",Centre-Forward,Colombia,"1,88 m",566.7 %,+€2.55m,161,48,7,28 / - / 1,8.733',"\n€3.00m Last update: Oct 3, 2023\n",https://www.transfermarkt.us/diber-cambindo/pr...,right,\n Attack - Centre-Forward ...,0,1x Top goal scorer
498,Luca Moro,\n #24,"Jan 25, 2001 (22)",Centre-Forward,Italy,"1,89 m",1020.0 %,+€2.55m,128,34,4,11 / - / -,8.030',"\n€2.80m Last update: Jun 12, 2023\n",https://www.transfermarkt.us/luca-moro/profil/...,"\n Jul 17, 2023 ...",right,0,1x Italienischer Zweitligameister


In [349]:
Firstdf['Achievemnts'] = Firstdf['Cups'].apply(clean_cups_column)
Firstdf=Firstdf.drop(["Cups"],axis=1)
Firstdf

,Name,Shirt Number,Age,Sub-Position,Country,Height,Matches,Goals,Assists,Cards,Minutes,Value,Links,Foot,Isfoot,Injury_Sum,Achievemnts
0,Erling Haaland,\n #9,"Jul 21, 2000 (23)",Centre-Forward,Norway,"1,95 m",253,202,48,23 / - / -,17.474',"\n€180.00m Last update: Jun 20, 2023\n",https://www.transfermarkt.us/erling-haaland/pr...,left,\n Attack - Centre-Forward ...,47,"1x UEFA Best Player in Europe, 5x Footballer o..."
1,Kylian Mbappé,\n #7,"Dec 20, 1998 (24)",Left Winger,France,"1,78 m",339,256,117,43 / - / 3,25.615',"\n€180.00m Last update: Jun 27, 2023\n",https://www.transfermarkt.us/kylian-mbappe/pro...,right,\n Attack - Left Winger ...,32,"3x Footballer of the Year, 10x Top goal scorer..."
2,Bukayo Saka,\n #7,"Sep 5, 2001 (22)",Right Winger,England,"1,78 m",248,66,64,24 / - / -,18.266',"\n€120.00m Last update: Jun 20, 2023\n",https://www.transfermarkt.us/bukayo-saka/profi...,"\n Jul 1, 2019 ...",left,6,"1x English FA Cup winner, 2x English Super Cup..."
3,Phil Foden,\n #47,"May 28, 2000 (23)",Left Winger,England,"1,71 m",276,86,58,8 / 1 / -,17.155',"\n€110.00m Last update: Jun 20, 2023\n",https://www.transfermarkt.us/phil-foden/profil...,left,\n Attack - Left Winger ...,26,"1x Champions League winner, 5x English Champio..."
4,Harry Kane,\n #9,"Jul 28, 1993 (30)",Centre-Forward,England,"1,88 m",529,326,87,48 / 1 / -,40.963',"\n€110.00m Last update: Oct 18, 2023\n",https://www.transfermarkt.us/harry-kane/profil...,right,\n Attack - Centre-Forward ...,80,"6x Top goal scorer, 4x Player of the Year, 1x ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Jon Karrikaburu,\n #9,"Sep 19, 2002 (21)",Centre-Forward,Spain,"1,83 m",80,23,4,11 / - / -,4.148',"\n€3.00m Last update: Oct 13, 2023\n",https://www.transfermarkt.us/jon-karrikaburu/p...,right,\n Attack - Centre-Forward ...,0,
496,Leo Sauer,\n #25,"Dec 16, 2005 (17)",Left Winger,Slovakia,"1,84 m",47,7,6,1 / - / -,2.907',"\n€3.00m Last update: Oct 17, 2023\n",https://www.transfermarkt.us/leo-sauer/profil/...,"\n Sep 1, 2023 ...",right,0,
497,Ilias Akhomach,\n #27,"Apr 16, 2004 (19)",Right Winger,Spain,"1,75 m",66,5,3,7 / 1 / 2,3.588',"\n€3.00m Last update: Jun 30, 2023\n",https://www.transfermarkt.us/ilias-akhomach/pr...,left,\n Attack - Right Winger ...,4,
498,Rodrigo Gomes,\n #21,"Jul 7, 2003 (20)",Right Winger,Portugal,"1,75 m",80,14,3,4 / - / -,4.442',"\n€3.00m Last update: Jun 16, 2023\n",https://www.transfermarkt.us/rodrigo-gomes/pro...,right,\n Attack - Right Winger ...,0,1x Portuguese cup winner


In [361]:
Firstdf.to_csv("Most.csv")
Valuedf.to_csv("Diff.csv")

# Scraping more for ML

In [2]:
Links=pd.read_csv("/Users/aa085/Music/Linkin.csv")
Links

,Unnamed: 0,Links
0,0,https://www.transfermarkt.us/jack-clarke/profi...
1,1,https://www.transfermarkt.us/luka-ivanusec/pro...
2,2,https://www.transfermarkt.us/michael-gregorits...
3,3,https://www.transfermarkt.us/isaiah-jones/prof...
4,4,https://www.transfermarkt.us/mathys-tel/profil...
...,...,...
737,737,https://www.transfermarkt.us/raphinha/profil/s...
738,738,https://www.transfermarkt.us/gabriel-strefezza...
739,739,https://www.transfermarkt.us/omar-marmoush/pro...
740,740,https://www.transfermarkt.us/firas-al-buraikan...


In [160]:
Mylist=Links["Links"]
Mylist=list(Mylist)
Mylist[0:5]

['https://www.transfermarkt.us/jack-clarke/profil/spieler/559794',
 'https://www.transfermarkt.us/luka-ivanusec/profil/spieler/387105',
 'https://www.transfermarkt.us/michael-gregoritsch/profil/spieler/120205',
 'https://www.transfermarkt.us/isaiah-jones/profil/spieler/696631',
 'https://www.transfermarkt.us/mathys-tel/profil/spieler/801734']

In [4]:
Ownlist2=[]

In [9]:
Mylist=Mylist[190:]

In [10]:
num_retries = 3  
retry_delay = 5  
user_agent = 'Your User-Agent'  

for url in tqdm(Mylist):
    for _ in range(num_retries):
        try:
            R = requests.get(url, headers={'User-Agent': 'Your User-Agent'})
            Soup = BeautifulSoup(R.text, "html.parser") 
            League=Soup.find("a",{"class":"data-header__league-link"}).text
            Team=Soup.find("span",{"class":"data-header__club"}).text

            mydict = {"League":League,
                      "Team":Team
                     }
            Ownlist2.append(mydict)
            break  

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except IndexError as e:
            print(f"Index error in data extraction: {e}")
            break  # Exit the loop if data structure is not as expected
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

 33%|█████████████▌                           | 183/552 [04:05<09:58,  1.62s/it]

Unexpected error: 'NoneType' object has no attribute 'text'


100%|█████████████████████████████████████████| 552/552 [12:31<00:00,  1.36s/it]


In [12]:
len(Ownlist2)

741

In [13]:
len(Links)

742

In [21]:
Links.iloc[371:375]["Links"]

371    https://www.transfermarkt.us/federico-bonazzol...
372    https://www.transfermarkt.us/ruben-garcia/prof...
373    https://www.transfermarkt.us/adnan-januzaj/pro...
374    https://www.transfermarkt.us/rony-lopes/profil...
Name: Links, dtype: object

In [20]:
Ownlist2[371:375]

[{'League': '\nSerie A                                    ',
  'Team': '\nHellas Verona '},
 {'League': '\nLaLiga                                    ',
  'Team': '\nSevilla FC '},
 {'League': '\nLiga Portugal                                    ',
  'Team': '\nSC Braga '},
 {'League': '\nSerie B                                    ',
  'Team': '\nParma '}]

In [24]:
Ownlist2[372]

{'League': 'LaLiga', 'Team': 'Osasuna'}

In [23]:
Ownlist2.insert(372, {'League': 'LaLiga', 'Team': 'Osasuna'})

In [31]:
for dictionary in Ownlist2:
    for key, value in dictionary.items():
            dictionary[key] = value.replace("\n", "")
Ownlist2[15:25]

[{'League': 'Ligue 1                                    ',
  'Team': 'Paris SG '},
 {'League': 'Bundesliga                                    ',
  'Team': 'Bayern Munich '},
 {'League': 'Premier League                                    ',
  'Team': 'Man Utd '},
 {'League': 'Ligue 1                                    ', 'Team': 'Monaco '},
 {'League': 'Ligue 1                                    ',
  'Team': 'FC Nantes '},
 {'League': 'Bundesliga                                    ',
  'Team': 'Bayern Munich '},
 {'League': 'Ligue 1                                    ',
  'Team': 'Paris SG '},
 {'League': 'Premier League                                    ',
  'Team': 'Liverpool '},
 {'League': 'Premier League                                    ',
  'Team': 'Aston Villa '},
 {'League': 'Bundesliga                                    ',
  'Team': 'Bayern Munich '}]

In [30]:
Newdf=pd.DataFrame(Ownlist2)
Newdf.iloc[371:375]

,League,Team
371,Serie A,Hellas Verona
372,LaLiga,Osasuna
373,LaLiga,Sevilla FC
374,Liga Portugal,SC Braga


In [32]:
Links

,Unnamed: 0,Links
0,0,https://www.transfermarkt.us/jack-clarke/profi...
1,1,https://www.transfermarkt.us/luka-ivanusec/pro...
2,2,https://www.transfermarkt.us/michael-gregorits...
3,3,https://www.transfermarkt.us/isaiah-jones/prof...
4,4,https://www.transfermarkt.us/mathys-tel/profil...
...,...,...
737,737,https://www.transfermarkt.us/raphinha/profil/s...
738,738,https://www.transfermarkt.us/gabriel-strefezza...
739,739,https://www.transfermarkt.us/omar-marmoush/pro...
740,740,https://www.transfermarkt.us/firas-al-buraikan...


In [152]:
Mylist=Links["Links"]
Mylist=list(Mylist)

In [153]:
for i,a in enumerate(Mylist):
    Mylist[i]=a.replace("profil","leistungsdatendetails")
Mylist[5:10]

['https://www.transfermarkt.us/ferran-torres/leistungsdatendetails/spieler/398184',
 'https://www.transfermarkt.us/jacob-murphy/leistungsdatendetails/spieler/199527',
 'https://www.transfermarkt.us/irfan-can-kahveci/leistungsdatendetails/spieler/216603',
 'https://www.transfermarkt.us/arnaud-kalimuendo/leistungsdatendetails/spieler/585959',
 'https://www.transfermarkt.us/zeki-amdouni/leistungsdatendetails/spieler/548729']

In [154]:
for i,a in enumerate(Mylist):
    Mylist[i]=a+"/plus/0?saison=2021&verein=&liga=&wettbewerb=&pos=&trainer_id="
Mylist[5:10]

['https://www.transfermarkt.us/ferran-torres/leistungsdatendetails/spieler/398184/plus/0?saison=2021&verein=&liga=&wettbewerb=&pos=&trainer_id=',
 'https://www.transfermarkt.us/jacob-murphy/leistungsdatendetails/spieler/199527/plus/0?saison=2021&verein=&liga=&wettbewerb=&pos=&trainer_id=',
 'https://www.transfermarkt.us/irfan-can-kahveci/leistungsdatendetails/spieler/216603/plus/0?saison=2021&verein=&liga=&wettbewerb=&pos=&trainer_id=',
 'https://www.transfermarkt.us/arnaud-kalimuendo/leistungsdatendetails/spieler/585959/plus/0?saison=2021&verein=&liga=&wettbewerb=&pos=&trainer_id=',
 'https://www.transfermarkt.us/zeki-amdouni/leistungsdatendetails/spieler/548729/plus/0?saison=2021&verein=&liga=&wettbewerb=&pos=&trainer_id=']

In [51]:
Ownlist2=[]

In [61]:
def scrape_player_stats(url, num_retries=3, retry_delay=5, user_agent='Your User-Agent'):
    for _ in range(num_retries):
        try:
            headers = {'User-Agent': user_agent}
            R = requests.get(url, headers=headers)
            Soup = BeautifulSoup(R.text, "html.parser")
            All = Soup.find_all("td", {"class": "zentriert"})
            Matches = All[1].text
            Goals = All[2].text
            Assists = All[3].text
            Minutes_elem = Soup.find_all("td", {"class": "rechts"})
            Minutes_elem=Minutes_elem[1]
            Minutes = Minutes_elem.text if Minutes_elem else "N/A"

            mydict = {
                "Matches": Matches,
                "Goals": Goals,
                "Assists": Assists,
                "Minutes": Minutes
            }
            Ownlist2.append(mydict)
            break

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except IndexError as e:
            print(f"Index error in data extraction: {e}")
            break  
        except Exception as e:
            print(f"Unexpected error: {e}")
            break

In [62]:
for url in tqdm(Mylist):
    scrape_player_stats(url)

 25%|██████████▏                              | 184/742 [07:20<19:06,  2.06s/it]

Index error in data extraction: list index out of range


 37%|███████████████▎                         | 277/742 [10:49<16:40,  2.15s/it]

Index error in data extraction: list index out of range


 65%|██████████████████████████▊              | 486/742 [19:15<05:55,  1.39s/it]

...

 92%|█████████████████████████████████████▋   | 682/742 [27:28<02:27,  2.45s/it]

Index error in data extraction: list index out of range


 94%|██████████████████████████████████████▍  | 695/742 [28:10<02:01,  2.59s/it]

Index error in data extraction: list index out of range


100%|█████████████████████████████████████████| 742/742 [30:09<00:00,  2.44s/it]


In [90]:
for i, a in enumerate(Ownlist2):
    if a == 0:
        Ownlist2[i] = {
            'Matches': '0',
            'Goals': '0',
            'Assists': '0',
            'Minutes': '0' 
        }

In [91]:
Ownlist2[183]

{'Matches': '0', 'Goals': '0', 'Assists': '0', 'Minutes': '0'}

In [92]:
data2021=pd.DataFrame(Ownlist2)

In [95]:
data2021=data2021.drop(["Assists"],axis=1)

In [96]:
data2021.rename(columns={'Matches': 'Goals', 'Goals': 'Assists'}, inplace=True)

In [97]:
data2021

,Goals,Assists,Minutes
0,5,9,2.776'
1,5,8,3.289'
2,9,1,1.541'
3,1,9,3.705'
4,9,-,799'
...,...,...,...
737,11,3,2.966'
738,14,6,2.777'
739,3,5,1.604'
740,11,4,2.199'


In [103]:
for i,a in enumerate(Mylist):
    Mylist[i]=a.replace("2021","2022")
Mylist[5:10]

['https://www.transfermarkt.us/ferran-torres/leistungsdatendetails/spieler/398184/plus/0?saison=2022&verein=&liga=&wettbewerb=&pos=&trainer_id=',
 'https://www.transfermarkt.us/jacob-murphy/leistungsdatendetails/spieler/199527/plus/0?saison=2022&verein=&liga=&wettbewerb=&pos=&trainer_id=',
 'https://www.transfermarkt.us/irfan-can-kahveci/leistungsdatendetails/spieler/216603/plus/0?saison=2022&verein=&liga=&wettbewerb=&pos=&trainer_id=',
 'https://www.transfermarkt.us/arnaud-kalimuendo/leistungsdatendetails/spieler/585959/plus/0?saison=2022&verein=&liga=&wettbewerb=&pos=&trainer_id=',
 'https://www.transfermarkt.us/zeki-amdouni/leistungsdatendetails/spieler/548729/plus/0?saison=2022&verein=&liga=&wettbewerb=&pos=&trainer_id=']

In [101]:
Ownlist2=[]

In [102]:
def scrape_player_stats(url, num_retries=3, retry_delay=5, user_agent='Your User-Agent'):
    for _ in range(num_retries):
        try:
            headers = {'User-Agent': user_agent}
            R = requests.get(url, headers=headers)
            Soup = BeautifulSoup(R.text, "html.parser")
            All = Soup.find_all("td", {"class": "zentriert"})
            Minutes_elem = Soup.find_all("td", {"class": "rechts"})
            
            if len(All) >= 3 and len(Minutes_elem) >= 1:
                Matches = All[0].text
                Goals = All[1].text
                Assists = All[2].text
                Minutes_elem = Minutes_elem[1]
                Minutes = Minutes_elem.text if Minutes_elem else "N/A"
            else:
                Matches = "0"
                Goals = "0"
                Assists = "0"
                Minutes = "0"

            mydict = {
                "Matches": Matches,
                "Goals": Goals,
                "Assists": Assists,
                "Minutes": Minutes
            }
            Ownlist2.append(mydict)
            break

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            time.sleep(retry_delay)
        except Exception as e:
            print(f"Unexpected error: {e}")
            break


In [104]:
for url in tqdm(Mylist):
    scrape_player_stats(url)

100%|█████████████████████████████████████████| 742/742 [30:28<00:00,  2.46s/it]


In [105]:
Ownlist2[500:505]

[{'Matches': '36', 'Goals': '18', 'Assists': '5', 'Minutes': "2.257'"},
 {'Matches': '33', 'Goals': '13', 'Assists': '8', 'Minutes': "2.478'"},
 {'Matches': '33', 'Goals': '7', 'Assists': '5', 'Minutes': "2.107'"},
 {'Matches': '23', 'Goals': '7', 'Assists': '5', 'Minutes': "1.655'"},
 {'Matches': '23', 'Goals': '-', 'Assists': '2', 'Minutes': "1.114'"}]

In [106]:
data2022=pd.DataFrame(Ownlist2)

In [107]:
data2022

,Matches,Goals,Assists,Minutes
0,50,11,14,4.179'
1,47,13,5,3.710'
2,42,15,5,2.860'
3,37,3,5,2.488'
4,28,6,-,600'
...,...,...,...,...
737,50,10,12,2.924'
738,36,9,4,2.582'
739,36,6,1,1.661'
740,32,18,4,2.499'


In [108]:
for i,a in enumerate(Mylist):
    Mylist[i]=a.replace("2022","2020")
Mylist[5:10]

['https://www.transfermarkt.us/ferran-torres/leistungsdatendetails/spieler/398184/plus/0?saison=2020&verein=&liga=&wettbewerb=&pos=&trainer_id=',
 'https://www.transfermarkt.us/jacob-murphy/leistungsdatendetails/spieler/199527/plus/0?saison=2020&verein=&liga=&wettbewerb=&pos=&trainer_id=',
 'https://www.transfermarkt.us/irfan-can-kahveci/leistungsdatendetails/spieler/216603/plus/0?saison=2020&verein=&liga=&wettbewerb=&pos=&trainer_id=',
 'https://www.transfermarkt.us/arnaud-kalimuendo/leistungsdatendetails/spieler/585959/plus/0?saison=2020&verein=&liga=&wettbewerb=&pos=&trainer_id=',
 'https://www.transfermarkt.us/zeki-amdouni/leistungsdatendetails/spieler/548729/plus/0?saison=2020&verein=&liga=&wettbewerb=&pos=&trainer_id=']

In [110]:
Ownlist2=[]

In [148]:
for url in tqdm(Mylist):
    scrape_player_stats(url)

100%|█████████████████████████████████████████| 426/426 [16:35<00:00,  2.34s/it]


In [151]:
data2020=pd.DataFrame(Ownlist2)
data2020

,Matches,Goals,Assists,Minutes
0,25,2,3,1.298'
1,48,9,5,3.052'
2,26,1,2,962'
3,23,2,6,1.551'
4,0,0,0,0
...,...,...,...,...
737,38,8,11,2.892'
738,31,4,-,1.912'
739,27,8,3,1.832'
740,15,2,-,463'


In [157]:
data2020.to_csv("Data20.csv")
data2021.to_csv("Data21.csv")
data2022.to_csv("Data22.csv")

In [159]:
Newdf.to_csv("Data0.csv")